Carlos Henrique Amorim Dutra

In [2]:
%%capture
# Local installation
!git clone https://github.com/speechbrain/speechbrain/
%cd /content/speechbrain/
!pip install -r requirements.txt
!pip install -e .

In [3]:
import speechbrain as sb

In [4]:
%cd /content/speechbrain/templates/speaker_id
!python train.py train.yaml --number_of_epochs=15 #--device='gpu'

/content/speechbrain/templates/speaker_id
rirs_noises.zip: 1.31GB [00:43, 29.9MB/s]                
Extracting ./data/rirs_noises.zip to ./data
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ./results/speaker_id/1986
train-clean-5.tar.gz: 333MB [00:13, 25.6MB/s]               
mini_librispeech_prepare - Creating train.json, valid.json, and test.json
mini_librispeech_prepare - train.json successfully created!
mini_librispeech_prepare - valid.json successfully created!
mini_librispeech_prepare - test.json successfully created!
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - 4.5M trainable parameters in SpkIdBrain
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going int

In [5]:
# Create folder for best model
!mkdir /content/best_model/

# Copy label encoder
!cp results/speaker_id/1986/save/label_encoder.txt /content/best_model/

# Copy best model
!cp "`ls -td results/speaker_id/1986/save/CKPT* | tail -1`"/* /content/best_model/

In [6]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier

classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb")
signal, fs =torchaudio.load('/content/speechbrain/tests/samples/single-mic/example1.wav')

# Compute speaker embeddings
embeddings = classifier.encode_batch(signal)

# Perform classification
output_probs, score, index, text_lab = classifier.classify_batch(signal)

# Posterior log probabilities
print(output_probs)

# Score (i.e, max log posteriors)
print(score)

# Index of the predicted speaker
print(index)

# Text label of the predicted speaker
print(text_lab)


tensor([[-31.8673, -35.2026, -25.7932,  ..., -21.0044, -12.4280, -21.5266]])
tensor([-1.1279])
tensor([2710])
['id10892']


/content/speechbrain/speechbrain/dataio/encoder.py:722: UserWarning: CategoricalEncoder.expect_len was never called: assuming category count of 7205 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct size. If you are sure this does not apply to you, use `.ignore_len`.
  warnings.warn(


For those of you interested in speaker verification, we also created an inference interface called `SpeakerRecognition`:

In [7]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

file1 = '/content/speechbrain/tests/samples/single-mic/example1.wav'
file2 = '/content/speechbrain/tests/samples/single-mic/example2.flac'

score, prediction = verification.verify_files(file1, file2)

print(score)
print(prediction) # True = same speaker, False=Different speakers

tensor([0.1799])
tensor([False])


In [8]:
%%writefile /content/best_model/hparams_inference.yaml

# #################################
# Basic inference parameters for speaker-id. We have first a network that
# computes some embeddings. On the top of that, we employ a classifier.
#
# Author:
#  * Mirco Ravanelli 2021
# #################################

# pretrain folders:
pretrained_path: /content/best_model/


# Model parameters
n_mels: 23
sample_rate: 16000
n_classes: 28 # In this case, we have 28 speakers
emb_dim: 512 # dimensionality of the embeddings

# Feature extraction
compute_features: !new:speechbrain.lobes.features.Fbank
    n_mels: !ref <n_mels>

# Mean and std normalization of the input features
mean_var_norm: !new:speechbrain.processing.features.InputNormalization
    norm_type: sentence
    std_norm: False

# To design a custom model, either just edit the simple CustomModel
# class that's listed here, or replace this `!new` call with a line
# pointing to a different file you've defined.
embedding_model: !new:custom_model.Xvector
    in_channels: !ref <n_mels>
    activation: !name:torch.nn.LeakyReLU
    tdnn_blocks: 5
    tdnn_channels: [512, 512, 512, 512, 1500]
    tdnn_kernel_sizes: [5, 3, 3, 1, 1]
    tdnn_dilations: [1, 2, 3, 1, 1]
    lin_neurons: !ref <emb_dim>

classifier: !new:custom_model.Classifier
    input_shape: [null, null, !ref <emb_dim>]
    activation: !name:torch.nn.LeakyReLU
    lin_blocks: 1
    lin_neurons: !ref <emb_dim>
    out_neurons: !ref <n_classes>

label_encoder: !new:speechbrain.dataio.encoder.CategoricalEncoder

# Objects in "modules" dict will have their parameters moved to the correct
# device, as well as having train()/eval() called on them by the Brain class.
modules:
    compute_features: !ref <compute_features>
    embedding_model: !ref <embedding_model>
    classifier: !ref <classifier>
    mean_var_norm: !ref <mean_var_norm>

pretrainer: !new:speechbrain.utils.parameter_transfer.Pretrainer
    loadables:
        embedding_model: !ref <embedding_model>
        classifier: !ref <classifier>
        label_encoder: !ref <label_encoder>
    paths:
        embedding_model: !ref <pretrained_path>/embedding_model.ckpt
        classifier: !ref <pretrained_path>/classifier.ckpt
        label_encoder: !ref <pretrained_path>/label_encoder.txt


Writing /content/best_model/hparams_inference.yaml


In [9]:
from speechbrain.pretrained import EncoderClassifier

classifier = EncoderClassifier.from_hparams(source="/content/best_model/", hparams_file='hparams_inference.yaml', savedir="/content/best_model/")

# Perform classification
audio_file = 'data/LibriSpeech/train-clean-5/5789/70653/5789-70653-0036.flac'
signal, fs = torchaudio.load(audio_file) # test_speaker: 5789
output_probs, score, index, text_lab = classifier.classify_batch(signal)
print('Target: 5789, Predicted: ' + text_lab[0])

# Another speaker
audio_file = 'data/LibriSpeech/train-clean-5/460/172359/460-172359-0012.flac'
signal, fs =torchaudio.load(audio_file) # test_speaker: 460
output_probs, score, index, text_lab = classifier.classify_batch(signal)
print('Target: 460, Predicted: ' + text_lab[0])

# And if you want to extract embeddings...
embeddings = classifier.encode_batch(signal)


/content/speechbrain/speechbrain/dataio/encoder.py:722: UserWarning: CategoricalEncoder.expect_len was never called: assuming category count of 28 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct size. If you are sure this does not apply to you, use `.ignore_len`.
  warnings.warn(


Target: 5789, Predicted: 5789
Target: 460, Predicted: 460


train1,train2,train3, train4 e train5 foram criados manualmente alterando o seed e alguns parametros

train1.yaml --number_of_epochs=5 --n_mels=10 --seed=1
--rnn_layers=3 --learning_rate=0.001

train2.yaml --number_of_epochs=5 --n_mels=20 --seed=2 --rnn_layers=2 --learning_rate=0.01

train3.yaml --number_of_epochs=10 --n_mels=15 --seed=3 --rnn_layers=4 --  noise_snr_high: 10

train4.yaml --number_of_epochs=15 --n_mels=12 --seed=321 --rnn_layers=2 --rnn_neurons=512 --learning_rate=0.0015

train5.yaml --number_of_epochs=20 --n_mels=18 --seed=5 --rnn_layers=3 --rnn_neurons=256 --learning_rate=0.002


In [1]:
##Com o comando abaixo, podemos executar vários treinamentos usando arquivos diferentes para cada parâmetro

%cd /content/speechbrain/templates/speaker_id

!python train.py train1.yaml

!python train.py train2.yaml

!python train.py train3.yaml

!python train.py train4.yaml

!python train.py train5.yaml

/content/speechbrain/templates/speaker_id
./data/rirs_noises.zip exists. Skipping download
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: ./results/speaker_id/1
mini_librispeech_prepare - Preparation completed in previous run, skipping.
speechbrain.dataio.encoder - Load called, but CategoricalEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - 4.5M trainable parameters in SpkIdBrain
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
  0% 0/76 [00:00<?, ?it/s]/content/speechbrain/speechbrain/dataio/encoder.py:722: UserWarning: CategoricalEncoder.expect_len was never called: assuming category count of 28 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct

In [4]:
# Create folder for best model
!mkdir /content/model1/

# Copy label encoder
!cp results/speaker_id/1/save/label_encoder.txt /content/model1/

# Copy best model
!cp "`ls -td results/speaker_id/1/save/CKPT* | tail -1`"/* /content/model1/

In [3]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="/content/speechbrain/templates/speaker_id/results/speaker_id/1/save/CKPT+2023-09-24+12-46-42+00")
signal, fs =torchaudio.load('/content/speechbrain/tests/samples/single-mic/example1.wav')

# Compute speaker embeddings
embeddings = classifier.encode_batch(signal)

# Perform classification
output_probs, score, index, text_lab = classifier.classify_batch(signal)

# Posterior log probabilities
print(output_probs)

# Score (i.e, max log posteriors)
print(score)

# Index of the predicted speaker
print(index)

# Text label of the predicted speaker
print(text_lab)
#nesse momento tive difuculdades para escolher o modelo

FileNotFoundError: ignored

Questão 1:
A primeira etapa do processo de criação de um modelo de reconhecimento de fala consiste na obtenção e tratamento dos dados, fase essa essencial para um bom modelo, pois remove ruídos, define como os dados vão ser estruturados, redução de dimensões e ampliação do dataset entre outras funções. A segunda fase é o treinamento, etapa essa mais demorada por ter vários processamentos diferentes incluídos e seu tempo de execução ser diretamente proporcional à quantidade de dados,começando pela extração de características (rasas ou profundas) e somente então a criação do modelo de identificação.

Questão 2:
O voiceprint seria uma analogia ao fingerprint (padrão único encontrado na nossa pele, que no caso seria o padrão único da voz), é a combinação de características que nos torna únicos. Algumas dessas características podem ser analisadas diretamente com com técnicas de machine learning.

Questão 3: existem dois tipos de extração de impressão de voz, o primeiro tipo pode ser analisado diretamente, isto é, não é necessário usar algoritmos de machine learning, estão em sua maioria no domínio do tempo e requer menor poder computacional. Exemplos: energia, pausas na fala . O outro tipo seria as informações mais profundas do áudio, geralmente no domínio da frequência, como MFCC, tom de voz, nasalidade e etc.

Questão 4:
O Objetivo de modelos de reconhecimento de voz são de comparar características obtidas por meio de algoritmos de modelo estatístico, isto é, podem ser utilizados em dataset fechados,onde o escore pode ser calculado. No caso de datasets abertas, os dados de entrada são considerados verdadeiros (a fala bate com o teste) ou não.

Questão 5:
Para calcular a precisão de uma aplicação de voz, deve-se usar os métodos já conhecidos em machine learning, como acurácia e precisão, baseados na matriz de confusão, onde os falsos positivos pelo total compõem o FAR e o FER que seria a taxa de falsos negativos. O EER é o ponto onde os dois valores são iguais.

Questão 6:

O speech brain funciona em três etapas, sendo elas a coleta de dados, treinamento do modelo e avaliação. O arquivo manifesto usa uma estrutura em Json ou sql para armazenar informações sobre o áudio, até emoções podem ser salvas na estrutura somente adicionando um novo label, onde a rede neural pode ser treinada para colher novas características do áudio para detecção de padrões que então podem ser usados para classificação de emoções por exemplo.

Questão 7:

Data sintesis e data augmentations são técnicas para multiplicar os dados disponíveis aumentando o dataset, a síntese de dados altera parâmetros físicos do dataset e o aumento de dados gera novos dados por meio de padrões, por exemplo a clipagem do mesmo áudio em ordem diferente separado pelos espaços sem som para criar novas frases.

Questão 8:
O aquivo de hiperparâmetros contem as configrações que a rede neural usará em seu treinamento para replicabilidade, se trata de um conjunto de variaveis, sua estrutura separa primeiramente a seed, como o computador não gera números totalmente aleatórios, os valores gerados estão atrelados sempre a uma seed. Além disso temos hiperparametros de modelo, do dataset, do treinamento, ruído e dos dados do resultado do treinamento, como a pasta de saída.


---

